In [1]:
cd ..

/Users/kaspar/Coding/Faculty/Projects/mindHacks-trafficking


In [2]:
import igraph as ig
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

#### Load Dataset

In [3]:
# Connect to the database
conn = sqlite3.connect("database/social_network_anonymized.db")

# Get the table names
cursor = conn.cursor()
query = "SELECT name FROM sqlite_master WHERE type='table';"
cursor.execute(query)
tables = cursor.fetchall()

df_profiles = pd.read_sql_query("SELECT * FROM Profiles", conn)
df_activity = pd.read_sql_query("SELECT * FROM Activity", conn)
df_media = pd.read_sql_query("SELECT * FROM Media", conn)
df_activity_media = pd.read_sql_query("SELECT * FROM ActivityMedia", conn)
df_profile_activity = pd.read_sql_query("SELECT * FROM ProfileActivity", conn)
df_profile_connection = pd.read_sql_query("SELECT * FROM ProfileConnection", conn)

### Convert to Network

#### Region basked breakdown

In [ ]:
persons = df_profiles[df_profiles['profile_type'] == 'person']
persons['region'].value_counts()

region
western       41251
thai           2080
vietnamese     1388
arabic         1074
chinese         677
indian          587
indonesian      294
korean          196
japanese         40
Name: count, dtype: int64

#### Convert to graph

In [5]:
g = ig.Graph(directed=True)
g.add_vertices(len(persons), attributes = {'bipartite':0})
g.vs['name'] = list(persons['name'])
g.vs['id'] = list(persons['id'])

##### Add Post count to Vertices

In [6]:
# Count 'posted-on-facebook' activities per profile
post_counts = df_profile_activity[df_profile_activity['activity_id'].isin(
    df_activity[df_activity['type'] == 'posted-on-facebook']['id']
)].groupby('profile_id').size()

# Add counts as vertex attribute
g.vs['post_count'] = 0
for profile_id, count in post_counts.items():
    matching_vertices = g.vs.select(id=profile_id)
    if matching_vertices:
        matching_vertices[0]['post_count'] = count


#### Get selected interactions

In [7]:
df_activity['type'].value_counts()
useful_activities = ['updated-friends-list-on-facebook', 'updated-followers-list-on-facebook', 'commented-on-facebook']
idxs = list(df_activity[df_activity['type'].isin(useful_activities)]['id'])
used_activity_profile_df = df_profile_activity[df_profile_activity['activity_id'].isin(idxs)]

#### Get indexes of source, target edges

In [8]:
# Group by activity_id and create tuples
result = []

for activity_id, group in tqdm(used_activity_profile_df.groupby('activity_id')):
    # Find source and target profiles
    source = group[group['relationship_type'] == 'source']['profile_id'].values
    target = group[group['relationship_type'] == 'target']['profile_id'].values
    
    # Only create tuples where both source and target exist
    if len(source) > 0 and len(target) > 0:
        for s in source:
            for t in target:
                result.append((activity_id, s.item(), t.item()))

# Convert to list of tuples
activity_links = result

# Preview results
print(f"Created {len(activity_links)} source-target relationships")
print(activity_links[:5])  # Show first 5 tuples

100%|██████████| 23328/23328 [00:17<00:00, 1321.40it/s]

Created 21764 source-target relationships
[(1, 27059, 24703), (2, 27059, 36779), (3, 27059, 8299), (4, 27059, 18673), (5, 27059, 14731)]


#### Add edges to graph

In [9]:
# Create a mapping from profile id to graph vertex index for quick lookups
id_to_index = {v['id']: i for i, v in enumerate(g.vs)}

# Add edges to the graph using id attribute
edges_added = 0
edges_skipped = 0

for activity_id, source_id, target_id in tqdm(activity_links):
    # Skip if either source or target id is not in the graph
    if source_id not in id_to_index or target_id not in id_to_index:
        edges_skipped += 1
        continue
        
    # Get activity type for edge label
    activity_type = df_activity.loc[df_activity['id'] == activity_id, 'type'].iloc[0]
    
    # Get vertex indices from id attribute
    source_index = id_to_index[source_id]
    target_index = id_to_index[target_id]
    
    # Add edge with attributes
    g.add_edge(
        source=source_index,
        target=target_index,
        activity_id=activity_id,
        label=activity_type
    )
    edges_added += 1

print(f"Added {edges_added} edges to the graph")
print(f"Skipped {edges_skipped} edges (node not found)")
print(f"Edge attributes: {g.es.attributes()}")

# Sample of edges with their labels
if g.es:
    edge_sample = [(g.vs[e.source]['id'], g.vs[e.target]['id'], e['label']) 
                  for e in g.es[:5]]
    print("Sample edges (source id, target id, type):", edge_sample)

  0%|          | 0/21764 [00:00<?, ?it/s]

100%|██████████| 21764/21764 [00:33<00:00, 649.96it/s] 

Added 20199 edges to the graph
Skipped 1565 edges (node not found)
Edge attributes: ['activity_id', 'label']
Sample edges (source id, target id, type): [(27059, 24703, 'updated-friends-list-on-facebook'), (27059, 36779, 'updated-friends-list-on-facebook'), (27059, 8299, 'updated-friends-list-on-facebook'), (27059, 14731, 'updated-friends-list-on-facebook'), (27059, 6086, 'updated-friends-list-on-facebook')]


#### Drop nodes that have no edges

In [10]:
# Identify isolated nodes (nodes with degree 0)
isolated_nodes = [v.index for v in g.vs if g.degree(v) == 0]

# Remove isolated nodes
g.delete_vertices(isolated_nodes)

print(f"Removed {len(isolated_nodes)} isolated nodes")
print(f"Graph now has {g.vcount()} nodes and {g.ecount()} edges")

Removed 32460 isolated nodes
Graph now has 15127 nodes and 20199 edges


#### Graph Breakdown:

In [11]:
import numpy as np

# Basic graph statistics
print(f"Vertices: {len(g.vs)}, Edges: {len(g.es)}")
print(f"Density: {g.density():.4f}")

# Connected components
components = g.components()
print(f"Number of connected components: {len(components)}")
print(f"Largest component size: {max(len(c) for c in components)}")
# print(f"Component size distribution: {np.bincount([len(c) for c in components])[1:]}")

# Node centrality measures
degree = g.degree()
betweenness = g.betweenness()
closeness = g.closeness()

# Find most central nodes
top_n = 5
top_degree_indices = np.argsort(degree)[-top_n:][::-1]
top_betweenness_indices = np.argsort(betweenness)[-top_n:][::-1]

print("\nTop nodes by degree centrality:")
for idx in top_degree_indices:
    print(f"ID: {g.vs[idx]['id']}, Degree: {degree[idx]}")

print("\nTop nodes by betweenness centrality:")
for idx in top_betweenness_indices:
    print(f"ID: {g.vs[idx]['id']}, Betweenness: {betweenness[idx]:.2f}")

# Edge type statistics
edge_types = [e['label'] for e in g.es]
edge_type_counts = {t: edge_types.count(t) for t in set(edge_types)}
print("\nEdge types distribution:")
for edge_type, count in sorted(edge_type_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{edge_type}: {count}")

# Structural analysis
print(f"\nDiameter (largest distance): {g.diameter()}")
print(f"Average path length: {g.average_path_length()}")
print(f"Transitivity (clustering): {g.transitivity_undirected():.4f}")

Vertices: 15127, Edges: 20199
Density: 0.0001
Number of connected components: 14145
Largest component size: 577

Top nodes by degree centrality:
ID: 18897, Degree: 5347
ID: 10786, Degree: 3218
ID: 27059, Degree: 3086
ID: 18631, Degree: 1931
ID: 10558, Degree: 1136

Top nodes by betweenness centrality:
ID: 18897, Betweenness: 12838356.13
ID: 6607, Betweenness: 8873801.12
ID: 31518, Betweenness: 8305176.11
ID: 27059, Betweenness: 6521603.27
ID: 24602, Betweenness: 5616785.81

Edge types distribution:
updated-friends-list-on-facebook: 10365
commented-on-facebook: 7645
updated-followers-list-on-facebook: 2189

Diameter (largest distance): 19
Average path length: 6.457100000498333
Transitivity (clustering): 0.0001


#### In vs Out edges

In [12]:
# Create subgraph with only 'commented-on-facebook' edges
comment_edges = [i for i, e in enumerate(g.es) if e['label'] == 'commented-on-facebook']
fb_comment_subgraph = g.subgraph_edges(comment_edges, delete_vertices=False)

# Calculate in-degree and out-degree for the subgraph
in_degree = fb_comment_subgraph.degree(mode="in")
out_degree = fb_comment_subgraph.degree(mode="out")

# Find top 10 nodes with highest in-degree (receiving comments)
top_in_indices = np.argsort(in_degree)[-10:][::-1]
print("Top 10 nodes receiving Facebook comments:")
for i, idx in enumerate(top_in_indices):
    print(f"{i+1}. ID: {g.vs[idx]['id']}, Comments received: {in_degree[idx]}")

# Find top 10 nodes with highest out-degree (commenting on others)
top_out_indices = np.argsort(out_degree)[-10:][::-1]
print("\nTop 10 nodes commenting on Facebook:")
for i, idx in enumerate(top_out_indices):
    print(f"{i+1}. ID: {g.vs[idx]['id']}, Comments made: {out_degree[idx]}")

# Calculate total comment activity
total_comments = len(comment_edges)
print(f"\nTotal Facebook comments in graph: {total_comments}")

# Optional: Calculate comment activity distribution
comment_counts = np.bincount(in_degree)
print(f"Distribution of comments received: {comment_counts[1:10]}")
print(f"Max comments received by any node: {max(in_degree)}")

Top 10 nodes receiving Facebook comments:
1. ID: 14765, Comments received: 660
2. ID: 18897, Comments received: 515
3. ID: 10786, Comments received: 423
4. ID: 16876, Comments received: 419
5. ID: 10558, Comments received: 323
6. ID: 27059, Comments received: 243
7. ID: 14833, Comments received: 120
8. ID: 31044, Comments received: 110
9. ID: 6231, Comments received: 108
10. ID: 30628, Comments received: 108

Top 10 nodes commenting on Facebook:
1. ID: 27059, Comments made: 364
2. ID: 18897, Comments made: 216
3. ID: 14765, Comments made: 128
4. ID: 30988, Comments made: 111
5. ID: 10558, Comments made: 76
6. ID: 16876, Comments made: 48
7. ID: 35339, Comments made: 40
8. ID: 18744, Comments made: 36
9. ID: 10943, Comments made: 35
10. ID: 14436, Comments made: 32

Total Facebook comments in graph: 7645
Distribution of comments received: [509 623  55 118  31  37  21  21  17]
Max comments received by any node: 660


#### Top by centrality

In [13]:
pr = g.betweenness()

#Top nodes:
top_n = 10
top_pr_indices = np.argsort(pr)[-top_n:][::-1]
print("\nTop nodes by PageRank:")
for idx in top_pr_indices:
    # Given a node index 'node_idx'
    in_comments = sum(1 for e in g.incident(idx, mode="in") if g.es[e]['label'] == 'commented-on-facebook')
    out_comments = sum(1 for e in g.incident(idx, mode="out") if g.es[e]['label'] == 'commented-on-facebook')
    print(f"ID: {g.vs[idx]['id']}, PageRank: {pr[idx]:.4f}", 'in',in_comments, 'out',out_comments)


Top nodes by PageRank:
ID: 18897, PageRank: 12838356.1324 in 515 out 216
ID: 6607, PageRank: 8873801.1176 in 6 out 10
ID: 31518, PageRank: 8305176.1087 in 30 out 31
ID: 27059, PageRank: 6521603.2669 in 243 out 364
ID: 24602, PageRank: 5616785.8096 in 22 out 26
ID: 10786, PageRank: 4603651.0000 in 423 out 0
ID: 12757, PageRank: 3193644.4224 in 2 out 5
ID: 14630, PageRank: 3055353.0212 in 0 out 1
ID: 10558, PageRank: 2977452.2400 in 323 out 76
ID: 8415, PageRank: 2540820.6554 in 15 out 6


#### Investigate single node

In [14]:
idx = 18897

act_idxs = used_activity_profile_df[(used_activity_profile_df['profile_id'] == idx) & (used_activity_profile_df['relationship_type'] == 'source')]['activity_id'].unique()


df_x = df_activity[df_activity['id'].isin(act_idxs)]
df_x = df_x[df_x['type'] == 'commented-on-facebook']
# act_idxs = df_x['activity_id'].unique()


for x in df_x[['content', 'description']].values:
    print(x[0])



Ajay Dutta  เพียงพอครับสำหรับ 1 คู่
Wasan Unthayanon สอบถามเพิ่มเติม แอดไลน์เบอร์นี้ครับ 0981616528
Tina Soriano
Tina Soriano
ถาวร พุทธรักษา ขอบคุณครับ
ถาวร พุทธรักษา ขอบคุณครับ
ถาวร พุทธรักษา ขอบคุณครับ
Eragon Wangkrapong เฉลี่ยประมาณ 3 ปีครับ คู่ล่างครอกที่แล้วสามตัวครับ
Eragon Wangkrapong เฉลี่ยประมาณ 3 ปีครับ คู่ล่างครอกที่แล้วสามตัวครับ
นกแก้ว ปาดี ไม่แรงครับ RAKA ตลาด ครับ
นกแก้ว ปาดี ไม่แรงครับ RAKA ตลาด ครับ
G'Golf Gab
G'Golf Gab
G'Golf Gab
Vincenzo Marcella Volante  35k
Lamont Martinez No, I can’t.
Lamont Martinez No, I can’t.
Lamont Martinez No, I can’t.
การะเกด ทิพยะมาศโกมล IB คับ
การะเกด ทิพยะมาศโกมล ขออนุญาตทักไปหานะครับ
การะเกด ทิพยะมาศโกมล ขอบคุณมากครับ
การะเกด ทิพยะมาศโกมล 218k ครับ
การะเกด ทิพยะมาศโกมล  แจ้งทางข้อความไปนะครับ
การะเกด ทิพยะมาศโกมล แจ้งทาง IB นะคับ
Ammini Minaka ส่งข้อความไปให้ทางอินบ๊อกซ์นะครับ
เก่ง หมอ เก้าคาร์แคร์ แจ้งทางอินบ๊อกซ์นะครับ
ชาติ ศิริเยี่ยม
Ben Bos White-lipped tamarin.
Neofelis Diardi DM
Neofelis Diardi DM
โศกราช ศรีธรรมราชธำรงฤทธิ์ 30 ใบ

#### Output a DataFrame

In [15]:
# Create dataframe with profile metrics
import pandas as pd

# Calculate metrics
betweenness_centrality = g.betweenness()
degree_centrality = g.degree()
pagerank = g.pagerank()
profile_ids = g.vs['id']
post_counts = g.vs['post_count']

# Calculate in/out comment counts for each node
in_comments = [sum(1 for e in g.incident(idx, mode="in") if g.es[e]['label'] == 'commented-on-facebook') 
               for idx in range(len(g.vs))]
out_comments = [sum(1 for e in g.incident(idx, mode="out") if g.es[e]['label'] == 'commented-on-facebook') 
                for idx in range(len(g.vs))]

friend_counts = [sum(1 for e in g.incident(idx) if g.es[e]['label'] == 'updated-friends-list-on-facebook') 
                for idx in range(len(g.vs))]

follower_counts = [sum(1 for e in g.incident(idx, mode="in") if g.es[e]['label'] == 'updated-followers-list-on-facebook') 
                  for idx in range(len(g.vs))]

following_counts = [sum(1 for e in g.incident(idx, mode="out") if g.es[e]['label'] == 'updated-followers-list-on-facebook') 
                   for idx in range(len(g.vs))]


# Create dataframe with all metrics
df_metrics = pd.DataFrame({
    'profile_id': profile_ids,
    'degree': degree_centrality,
    'pagerank': pagerank,
    'betweenness': betweenness_centrality,
    'comments_received': in_comments,
    'comments_made': out_comments,
    'posts': post_counts,
    'friends': friend_counts,
    'followers': follower_counts,
    'following': following_counts
})

# Add rank and sort
df_metrics['centrality_rank'] = df_metrics['betweenness'].rank(ascending=False)
df_metrics = df_metrics.sort_values('betweenness', ascending=False).reset_index(drop=True)

# Show top profiles
df_metrics.head(10)

,profile_id,degree,pagerank,betweenness,comments_received,comments_made,posts,friends,followers,following,centrality_rank
0,18897,5347,0.007673,1.283836e+07,515,216,294,4540,75,0,1.0
1,6607,17,0.000097,8.873801e+06,6,10,0,0,0,0,2.0
2,31518,61,0.000664,8.305176e+06,30,31,3,0,0,0,3.0
3,27059,3086,0.006954,6.521603e+06,243,364,224,2340,138,0,4.0
4,24602,48,0.000684,5.616786e+06,22,26,59,0,0,0,5.0
5,10786,3218,0.009897,4.603651e+06,423,0,88,2748,47,0,6.0
6,12757,7,0.000072,3.193644e+06,2,5,0,0,0,0,7.0
7,14630,2,0.000031,3.055353e+06,0,1,0,0,0,0,8.0
8,10558,1136,0.005349,2.977452e+06,323,76,22,737,0,0,9.0
9,8415,21,0.000459,2.540821e+06,15,6,2,0,0,0,10.0


#### Save the csv

In [16]:
df_metrics.to_csv('profile_metrics.csv', index=False)